# Machine Learning Pipeline - Feature Selection

In this notebook, we pick up the transformed datasets that we saved in the previous notebook.

## Reproducibility: Setting the seed

With the aim to ensure reproducibility between runs of the same notebook, but also between the research and production environment, for each step that includes some element of randomness, it is extremely important that we **set the seed**.

In [1]:
# to handle datasets
import pandas as pd
import numpy as np

# for plotting
import matplotlib.pyplot as plt

# to build the models
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

In [5]:
# cargar el conjunto de entrenamiento y prueba con las engineered variables

# Nosotros construimos y guardamos estos conjuntos de datos en la lección anterior.
# Si no lo has hecho, revisa el NOTEBOOK anterior para saber cómo crear estos conjuntos de datos.

X_train = pd.read_csv('./Output/xtrain.csv')
X_test = pd.read_csv('./Output/xtest.csv')

X_train.head()

,C3S,Retenido M450,Fraguado final,Superficie específica,CaCO3,Retenido M325,Yeso,Expansión en autoclave,Fe2O3,Na2O,Polvo Filtro,MgO,SO3,Cal Libre,Fluidez
0,0.475299,0.722222,0.500000,0.465517,0.236599,0.500000,0.538462,0.0,0.480769,0.727273,0.0,0.666667,0.500000,0.666667,0.000000
1,0.155486,0.666667,0.357143,0.793103,0.683919,1.000000,0.250000,0.0,0.173077,0.454545,0.0,0.333333,0.452381,0.000000,0.285714
2,0.165367,0.611111,0.785714,0.655172,0.451017,0.500000,0.826923,0.0,0.519231,0.636364,0.0,0.333333,0.571429,0.666667,0.714286
3,0.520541,0.388889,0.142857,0.482759,0.421442,0.500000,0.288462,0.0,0.538462,0.454545,0.0,0.333333,0.476190,1.000000,0.142857
4,0.421737,0.444444,0.571429,0.698276,0.255083,0.666667,0.548077,0.5,0.480769,0.636364,0.0,0.666667,0.238095,0.000000,0.428571


In [6]:
# Cargar el objetivo (recuerda que el objetivo está transformado logarítmicamente).
y_train = pd.read_csv('./Output/ytrain.csv')
y_test = pd.read_csv('./Output/ytest.csv')

y_train.head()

,RC 1 día
0,7.251345
1,7.244228
2,7.265430
3,7.565275
4,7.170120


### Feature Selection

Vamos a seleccionar un subconjunto de las características más predictivas. Hay un elemento de aleatoriedad en la regresión Lasso, así que recuerda establecer la semilla.

In [7]:
# Realizaremos el ajuste del modelo y la selección de características
# todo junto en unas pocas líneas de código

# primero, especificamos el modelo de Regresión Lasso y
# seleccionamos un alpha adecuado (equivalente a la penalización).
# Cuanto mayor sea el alpha, menos características serán seleccionadas.

# Luego usamos el objeto selectFromModel de sklearn, que
# seleccionará automáticamente las características cuyos coeficientes sean diferentes de cero

# recuerda establecer la semilla (random_state), el estado aleatorio en esta función
sel_ = SelectFromModel(Lasso(alpha=0.001, random_state=0))

# Entrenamos el Modelo Lasso y seleccionamos los features
sel_.fit(X_train, y_train)

SelectFromModel(estimator=Lasso(alpha=0.001, random_state=0))

Cantidad de Características seleccionadas:

In [8]:
sel_.get_support().sum()

10

In [9]:
# Visualicemos aquellas características que fueron seleccionadas.
# (características seleccionadas marcadas con True)

sel_.get_support()

array([ True,  True,  True,  True, False,  True,  True,  True, False,
       False,  True, False,  True,  True, False])

In [10]:
# Imprimamos el número de características totales y seleccionadas

# Así es como podemos hacer una lista de las características seleccionadas
selected_feats = X_train.columns[(sel_.get_support())]

# Imprimamos algunas estadísticas
print('Número de Features en Total: {}'.format((X_train.shape[1])))
print('Número de Features Seleccionados: {}'.format(len(selected_feats)))
print('Características con coeficientes reducidos a cero: {}'.format(
    np.sum(sel_.estimator_.coef_ == 0)))

Número de Features en Total: 15
Número de Features Seleccionados: 10
Características con coeficientes reducidos a cero: 5


In [11]:
# Imprimimos las Características o Features seleccionados
selected_feats

Index(['C3S', 'Retenido M450', 'Fraguado final', 'Superficie específica',
       'Retenido M325', 'Yeso', 'Expansión en autoclave', 'Polvo Filtro',
       'SO3', 'Cal Libre'],
      dtype='object')

In [12]:
pd.Series(selected_feats).to_csv('./Output/selected_features.csv', index=False)